# MHW Event 3D

In [1]:
# imports
import numpy as np
import os
from matplotlib import pyplot as plt

from datetime import date

import pandas
import sqlalchemy
from mayavi import mlab

# Load

In [2]:
mhw_file = '/home/xavier/Projects/Oceanography/MHW/db/mhws_allsky_defaults.db'
tst_file = '/home/xavier/Projects/Oceanography/MHW/db/test_mhws_allsky.db'
#
engine = sqlalchemy.create_engine('sqlite:///'+mhw_file)
connection = engine.connect()
connection

In [3]:
mhw_events = pandas.read_sql_table('MHW_Events', con=engine, 
                               columns=['date', 'lon', 'lat', 'duration' , 
                                        'ievent', 'time_peak', 'time_start'])
mhw_events.head()

,date,lon,lat,duration,ievent,time_peak,time_start
0,1982-01-01,0.125,-71.375,13,0,723552,723546
1,1982-12-20,0.125,-71.375,33,1,723901,723899
2,1985-01-30,0.125,-71.375,10,2,724677,724671
3,1988-01-10,0.125,-71.375,9,3,725752,725746
4,1991-01-09,0.125,-71.375,52,4,726858,726841


In [4]:
mhw_events = mhw_events.set_index('date')
mhw_events.head()

,lon,lat,duration,ievent,time_peak,time_start
date,,,,,,
1982-01-01,0.125,-71.375,13,0,723552,723546
1982-12-20,0.125,-71.375,33,1,723901,723899
1985-01-30,0.125,-71.375,10,2,724677,724671
1988-01-10,0.125,-71.375,9,3,725752,725746
1991-01-09,0.125,-71.375,52,4,726858,726841


## Add start date

In [5]:
mhw_events['peak_date'] = pandas.to_datetime([date.fromordinal(tt) for tt in mhw_events['time_peak']])

In [6]:
mhw_events.head()

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
1982-01-01,0.125,-71.375,13,0,723552,723546,1982-01-07
1982-12-20,0.125,-71.375,33,1,723901,723899,1982-12-22
1985-01-30,0.125,-71.375,10,2,724677,724671,1985-02-05
1988-01-10,0.125,-71.375,9,3,725752,725746,1988-01-16
1991-01-09,0.125,-71.375,52,4,726858,726841,1991-01-26


# Pick one

## Not so far from the equator

In [14]:
equ_events = mhw_events[np.abs(mhw_events['lat']) < 20.]

In [15]:
equ_events.iloc[np.argsort(equ_events['duration'])[-20:]]

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
2015-03-21,190.375,-2.125,433,55,736005,735678,2016-02-11
2015-03-20,190.125,-2.375,434,51,735997,735677,2016-02-03
2015-03-20,190.125,-2.625,434,51,736015,735677,2016-02-21
1997-06-27,270.625,-11.875,436,9,729351,729202,1997-11-23
1997-08-20,262.375,-10.875,436,10,729459,729256,1998-03-11
1997-08-19,262.625,-10.625,436,12,729417,729255,1998-01-28
1997-06-27,270.625,-12.125,436,11,729350,729202,1997-11-22
2015-03-13,219.875,4.875,438,69,735727,735670,2015-05-09
1997-06-25,270.625,-11.625,438,13,729351,729200,1997-11-23


In [16]:
ievent = np.argsort(equ_events['duration'])[-20:][0]
ievent

8385907

## Pacific Ocean

In [45]:
pacific_events = mhw_events[(np.abs(mhw_events['lat']-35.) < 15.) & (np.abs(mhw_events['lon']-238.) < 15.)]

In [48]:
pacific_events.iloc[np.argsort(pacific_events['duration'])[-1000:-950]]

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
2014-10-13,239.875,28.125,255,74,735674,735519,2015-03-17
2014-08-07,227.625,36.125,255,64,735685,735452,2015-03-28
2014-12-30,229.375,42.625,256,59,735779,735597,2015-06-30
2015-03-07,237.375,21.375,256,67,735839,735664,2015-08-29
2014-10-05,241.375,30.625,256,71,735634,735511,2015-02-05
2014-12-30,229.375,42.875,256,65,735791,735597,2015-07-12
2015-07-19,246.375,26.375,256,59,735926,735798,2015-11-24
2014-12-03,223.125,48.875,256,54,735750,735570,2015-06-01
2014-10-15,238.125,25.625,256,76,735730,735521,2015-05-12


In [51]:
pevent = np.argsort(pacific_events['duration'])[-955:][0]
pevent

655363

## Build a Cube

### Cut out near my event

In [18]:
eq_event = equ_events.iloc[ievent:ievent+1]
eq_event

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
2015-03-21,190.375,-2.125,433,55,736005,735678,2016-02-11


In [21]:
start_date = eq_event.index.values[0]
start_date

numpy.datetime64('2015-03-21T00:00:00.000000000')

In [22]:
end_date= start_date + pandas.Timedelta(days=eq_event.duration.values[0])
end_date

Timestamp('2016-05-27 00:00:00')

### Pacific

In [52]:
p_event = pacific_events.iloc[pevent:pevent+1]
p_event
#
p_start_date = p_event.index.values[0]
#
p_end_date= p_start_date + pandas.Timedelta(days=p_event.duration.values[0])
#
p_event, p_start_date, p_end_date

(                lon     lat  duration  ievent  time_peak  time_start  \
 date                                                                   
 2014-10-03  241.375  30.125       258      59     735571      735509   
 
             peak_date  
 date                   
 2014-10-03 2014-12-04  ,
 numpy.datetime64('2014-10-03T00:00:00.000000000'),
 Timestamp('2015-06-18 00:00:00'))

In [53]:
p_event.duration.values[0]

258

## Just use start for now

### Time

In [54]:
#neartime_events = mhw_events["2018-10-15":"2019-02-23"]
#neartime_events = equ_events["2015-03-21":"2016-05-27"]
neartime_events = pacific_events["2014-09-20":"2015-07-01"] # Buffered by ~10 days

In [55]:
neartime_events

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
2014-10-02,223.125,20.125,7,73,735509,735508,2014-10-03
2015-01-08,223.125,20.125,118,74,735637,735606,2015-02-08
2015-05-27,223.125,20.125,6,75,735747,735745,2015-05-29
2015-06-09,223.125,20.125,15,76,735760,735758,2015-06-11
2015-01-10,223.125,20.375,116,71,735637,735608,2015-02-08
...,...,...,...,...,...,...,...
2014-11-11,252.875,24.125,8,75,735554,735548,2014-11-17
2015-01-25,252.875,24.125,6,76,735627,735623,2015-01-29
2015-02-06,252.875,24.125,28,77,735645,735635,2015-02-16


### Space

In [56]:
event = p_event
#event.lat.values[0]
event.lat.values[0]

30.125

In [57]:
nearst_events = neartime_events[(np.abs(
    neartime_events.lat-event.lat.values[0]) < 10.) & (np.abs(
    neartime_events.lon-event.lon.values[0]) < 10.)]
nearst_events.head()

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
2014-10-28,231.625,20.375,5,67,735536,735534,2014-10-30
2014-11-28,231.625,20.375,15,68,735569,735565,2014-12-02
2015-01-11,231.625,20.375,39,69,735638,735609,2015-02-09
2015-02-25,231.625,20.375,21,70,735666,735654,2015-03-09
2015-03-28,231.625,20.375,10,71,735692,735685,2015-04-04


### Cube

In [58]:
event

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
2014-10-03,241.375,30.125,258,59,735571,735509,2014-12-04


In [59]:
deg = 10.
step = 0.25
grid_lon = np.arange(event.lon.values[0]-deg, event.lon.values[0]+deg+step, step)
grid_lat = np.arange(event.lat.values[0]-deg, event.lat.values[0]+deg+step, step)

In [60]:
cube = np.zeros((event.duration.values[0]+20, grid_lat.size, grid_lon.size), dtype='int')
cube.shape

(278, 81, 81)

In [61]:
for index, row in nearst_events.iterrows():
    #import pdb; pdb.set_trace()
    ilat = np.argmin(np.abs(grid_lat-row.lat))
    jlon = np.argmin(np.abs(grid_lon-row.lon))
    # Do it!
    t0 = row.time_start-event.time_start.values[0]+15
    if t0 < 0:
        import pdb; pdb.set_trace()
    cube[t0:t0+row.duration+1, ilat, jlon] = 1

In [62]:
np.sum(cube)/cube.size

0.3466472363946977

In [63]:
cube.size

1823958

## Viz

In [22]:
x, y, z = np.ogrid[-10:10:20j, -10:10:20j, -10:10:20j]
s = np.sin(x*y*z)/(x*y*z)

In [23]:
#mlab.contour3d(s)

### Save

In [64]:
#np.save('tst_cube.npy', cube)
np.save('tst_cube_pacific.npy', cube)